In [106]:
from bs4 import BeautifulSoup as bs 
import requests
from splinter import Browser
from urllib.parse import urljoin
import pandas as pd
from pprint import pprint
import time
import pymongo
import pprint

In [2]:
def latest_headline(): #grab the latest headline and teaser paragraph
    #url to scrape
    url= "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

    executable_path = {'executable_path': 'chromedriver'} #path to chromedriver; in this case it's in the same file
    browser = Browser('chrome', **executable_path, headless = False) #gives us a broswer object

    browser.visit(url)#visit url

    html_code = browser.html #get html text from page
    soup = bs(html_code, "html.parser") #beautiful soup to parse
    
    news_title = soup.find_all("div", class_='content_title')[0].text #title of first article 
    news_p = soup.find_all('div', class_="article_teaser_body")[0].text #paragraph of first article
    
    browser.quit()
    return {news_title: news_p} #return a dict of title and paragraph
    
    
current_headline=latest_headline() 

In [3]:
current_headline

{'MarCO Makes Space for Small Explorers': 'A pair of NASA CubeSats flying to Mars are opening a new frontier for small spacecraft.'}

In [84]:
def mars_img():
    url= "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

    executable_path = {'executable_path': 'chromedriver'} #path to chromedriver; in this case it's in the same file
    browser = Browser('chrome', **executable_path, headless = False) #gives us a broswer object

    browser.visit(url)#visit url

    html_code = browser.html #get html text from page
    soup = bs(html_code, "html.parser") #beautiful soup to parse

    button=browser.find_by_id('full_image') #find button
    button.click() #click button

    html2=browser.html #get html from new page
    soup2 = bs(html2, "html.parser") #beautiful soup to parse new page

    browser.quit() #quit browser

    class_containing_img=soup2.find('a',class_='button fancybox')['data-fancybox-href'] #gets relative image link "/img.jpg"

    return urljoin(url, class_containing_img) #join base url and relative url

latest_img=mars_img() #function call to get complete url

In [85]:
latest_img

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16217_ip.jpg'

In [ ]:
//*[@id="fancybox-lock"]/div/div[1]/img

In [6]:
def mars_tweet():
    url= "https://twitter.com/marswxreport?lang=en"

    executable_path = {'executable_path': 'chromedriver'} #path to chromedriver; in this case it's in the same file
    browser = Browser('chrome', **executable_path, headless = False) #gives us a broswer object

    browser.visit(url)#visit url

    html_code = browser.html #get html text from page
    soup = bs(html_code, "html.parser") #beautiful soup to parse

    browser.quit() #quit browser
    
    return soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text #get first tweet; text only

mars_weather=mars_tweet() #function call to get tweet string

In [7]:
mars_weather

'Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59'

In [8]:
def mars_table():
    url= "https://space-facts.com/mars/" #url to scrape

    tables= pd.read_html(url) #read html
    df=pd.DataFrame(tables[0]) #convert all tables to dfs ;only select the first table [0]

    return df.to_html() #return html

mars_table_html=mars_table() #function call to create html

In [9]:
mars_table_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    

In [88]:

url= "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

executable_path = {'executable_path': 'chromedriver'} #path to chromedriver; in this case it's in the same file
browser = Browser('chrome', **executable_path, headless = False) #gives us a broswer object

browser.visit(url)#visit url

html_code = browser.html #get html text from page
soup = bs(html_code, "html.parser") #beautiful soup to parse

data = soup.find('div', class_='collapsible results').find_all('div', class_='item')

data_dict = []
for i in range(len(data)):
    title = data[i].find('div', class_="description").find('h3').text
    browser.find_by_css('div[class="collapsible results"]').find_by_css('div[class="item"]')[i].find_by_css('div[class="description"]').find_by_css('a').click()
    for img in browser.find_by_css('div[class="downloads"]').find_by_css('a'):
        if ('Sample' in img.text):
            img_url = img['href']

    browser.back()
    mydict = {'title': title, 'img_url': img_url}
    data_dict.append(mydict)
    print(title)
    print(img_url)
    
# x=soup.find_all('div', class_='description').find
# x=browser.find_by_css('div[class="description"]').find_by_css('a')

# buttons=x.find_by_css('a[class="itemLink product-item"]') #find button; finds list of urls to click 

# z=browser.find_link_by_href('Cerberus')
# y=browser.find_by_xpath("//div[@class='description']/a")

# y[0].click()
# browser.back()
# y[1].click()

#     time.sleep(5)
#     browser.back()
# hemisphere_img_urls =[]

# for button in buttons:
    
#     button.click() #click button

#     html_code2 = browser.html #get html text from page
#     soup = bs(html_code3, "html.parser") #beautiful soup to parse

#     x=soup.find_all('h2', class_='title')[0].text
#     y=soup.find_all('a', target="_blank")[1].attrs['href']

#     mydict={}
#     mydict['title']=title
#     mydict['img_url']= img_url

#     hemisphere_img_urls.append(mydict)

# hemisphere_img_urls



Cerberus Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


In [89]:
data_dict

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [40]:
for i in hemispheres:
    print(i)
    print()

<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>

<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/7677c0a006b83871b5a2f66985ab5857_schiapare

In [ ]:
for i in x:
    print(i.html)
    print()

In [ ]:
# x[0].html

for idx, val in enumerate(x):
    
    print(idx, val.find_by_css('a[class="itemLink product-item"]').html)
    
# print(x[0].html)
# print()
# print(x[1].html)
# print()
# print(x[2].html)
# print()
# print(x[3].html)
# print()


In [ ]:
buttons[1]

In [55]:
mydict={'headline': {'MarCO Makes Space for Small Explorers': 'A pair of NASA CubeSats flying to Mars are opening a new frontier for small spacecraft.'}, 'img': 'https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA18884', 'hemis': [{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}], 'twit': 'Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59', 'table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'}

In [3]:
mydict

{'headline': {'MarCO Makes Space for Small Explorers': 'A pair of NASA CubeSats flying to Mars are opening a new frontier for small spacecraft.'},
 'img': 'https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA18884',
 'hemis': [{'title': 'Cerberus Hemisphere Enhanced',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
  {'title': 'Schiaparelli Hemisphere Enhanced',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
  {'title': 'Syrtis Major Hemisphere Enhanced',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
  {'title': 'Valles Marineris Hemisphere Enhanced',
   'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}],
 'twit': 'Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59',
 'table': '<table border="1" class="dataframe">\n  

In [57]:
mydict['hemis'][0]['img_url']

'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'

In [6]:
mydict['twit']

'Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59'

In [92]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.mars_app

In [93]:
client.database_names()

['Dumpster_DB',
 'admin',
 'config',
 'craigslist_app',
 'craigslist_db',
 'local',
 'mars_app',
 'movies',
 'team_db',
 'travel_db']

In [38]:
post = {
    'vendor': 'fruit star',
    'fruit': 'raspberry',
    'quantity': 21,
    'ripeness': 2
}


In [132]:
cursor = db.collection.find_one()  #find({})
for document in cursor: 
    pprint.pprint(document)

'_id'
'headline'
'img'
'hemis'
'twit'


In [131]:
a=cursor.collection.find_one
pprint.pprint(a)

<bound method Collection.find_one of Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mars_app'), 'collection')>


In [51]:
client.database_names()

['Dumpster_DB',
 'admin',
 'config',
 'craigslist_app',
 'craigslist_db',
 'local',
 'mars_app',
 'movies',
 'team_db',
 'travel_db']

In [52]:
client.drop_database('mars_app')

In [48]:
print(db.mars_app.find_one())


None


In [90]:
app.config["MONGO_URI"] = "mongodb://localhost:27017/mars_app"
mongo = PyMongo(app)

NameError: name 'app' is not defined